In [ ]:
import os
from PIL import Image
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.datasets as dsets
import torchvision

In [ ]:
import torch.nn as nn
import torchvision.models as models
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
training_path = "/content/drive/MyDrive/data/Training"
testing_path = "/content/drive/MyDrive/data/Testing"

In [ ]:
for label, subdir in enumerate([d for d in os.listdir(training_path) if not d.startswith(".")]):
  print(label, subdir)

0 meningioma_tumor
1 no_tumor
2 pituitary_tumor
3 glioma_tumor


In [ ]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(299),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), # ToTensor : [0, 255] -> [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(), # ToTensor : [0, 255] -> [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
train_data = dsets.ImageFolder(training_path, train_transform)
test_data = dsets.ImageFolder(testing_path, test_transform)

In [ ]:
batch_size = 32

train_loader = DataLoader(train_data,
                          batch_size=batch_size,
                          shuffle=True)

test_loader = DataLoader(test_data,
                         batch_size=batch_size,
                         shuffle=True)

In [ ]:
def imshow(img, title):
    img = torchvision.utils.make_grid(img, normalize=True)
    npimg = img.numpy()
    fig = plt.figure(figsize = (5, 15))
    plt.imshow(np.transpose(npimg,(1,2,0)))
    plt.title(title)
    plt.axis('off')
    plt.show()

In [ ]:
# for images, labels in train_loader:
    # imshow(images, [train_data.classes[i] for i in labels])


In [ ]:
model = models.inception_v3(pretrained=True)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:01<00:00, 105MB/s]


In [ ]:
model.aux_logits = False

for parameter in model.parameters():
    parameter.requires_grad = False

In [ ]:
num_classes = 4

In [ ]:
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 10),
    nn.Linear(10, num_classes)
)

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

In [ ]:
num_epochs = 10

In [17]:
# for epoch in range(num_epochs):
#     running_loss = 0.0
#     for i, data in enumerate(dataloader, 0):
#         inputs, labels = data
#         optimizer.zero_grad()

#         outputs = inception_model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#     print(f"Epoch {epoch + 1}, Loss: {running_loss / len(dataloader)}")



for epoch in range(num_epochs):

    total_batch = len(train_data)//batch_size

    for i, (batch_images, batch_labels) in enumerate(train_loader):

        X = batch_images
        Y = batch_labels

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 5 == 0:
            print('Epoch [%d/%d], lter [%d/%d] Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

Epoch [1/10], lter [5/90] Loss: 2.3723
Epoch [1/10], lter [10/90] Loss: 1.6926
Epoch [1/10], lter [15/90] Loss: 1.4887
Epoch [1/10], lter [20/90] Loss: 1.3906
Epoch [1/10], lter [25/90] Loss: 1.1902
Epoch [1/10], lter [30/90] Loss: 1.1837
Epoch [1/10], lter [35/90] Loss: 1.2441
Epoch [1/10], lter [40/90] Loss: 1.0520
Epoch [1/10], lter [45/90] Loss: 0.9678
Epoch [1/10], lter [50/90] Loss: 1.1073
Epoch [1/10], lter [55/90] Loss: 1.1574
Epoch [1/10], lter [60/90] Loss: 1.3856
Epoch [1/10], lter [65/90] Loss: 1.1837
Epoch [1/10], lter [70/90] Loss: 1.2594
Epoch [1/10], lter [75/90] Loss: 0.9314
Epoch [1/10], lter [80/90] Loss: 1.0943
Epoch [1/10], lter [85/90] Loss: 0.8734
Epoch [1/10], lter [90/90] Loss: 0.9026
Epoch [2/10], lter [5/90] Loss: 0.9373
Epoch [2/10], lter [10/90] Loss: 0.8090
Epoch [2/10], lter [15/90] Loss: 0.9055
Epoch [2/10], lter [20/90] Loss: 0.9546
Epoch [2/10], lter [25/90] Loss: 0.7598
Epoch [2/10], lter [30/90] Loss: 0.8854
Epoch [2/10], lter [35/90] Loss: 1.0623
Ep

KeyboardInterrupt: 

In [18]:
model.eval()

correct = 0
total = 0

for images, labels in test_loader:

    images = images
    outputs = model(images)

    _, predicted = torch.max(outputs.data, 1)

    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of test images: %f %%' % (100 * float(correct) / total))

Accuracy of test images: 52.791878 %
